In [26]:
from pyspark.sql import SparkSession

In [27]:
spark = SparkSession.builder.appName('spark-lab').getOrCreate()

In [28]:
spark

In [29]:
url = "jdbc:postgresql://containers-us-west-142.railway.app:7760/railway"
properties = {
    "user": "postgres",
    "password": "",
    "driver": "org.postgresql.Driver"
}


In [32]:
df = spark.read.format("jdbc").option("url", url) \
    .option("driver", "org.postgresql.Driver").option("dbtable", "data") \
    .option("user", properties['user']).option("password", properties['password']).load()

In [33]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- pin-code: integer (nullable = true)



In [34]:
df.show()

+---+-------------+------+--------+
| id|         name|  city|pin-code|
+---+-------------+------+--------+
|  1|Gaurav Sharma|Mohali|  177041|
+---+-------------+------+--------+



In [37]:
df.filter(df['pin-code'] == 177041).show()

+---+-------------+------+--------+
| id|         name|  city|pin-code|
+---+-------------+------+--------+
|  1|Gaurav Sharma|Mohali|  177041|
+---+-------------+------+--------+

